In [15]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [35]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
image, label = train_data[0]

In [13]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [29]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x =  self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x      
    

In [36]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [37]:
for epoch in range(30):
    print(f"Training epoch {epoch}...")
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss =+ loss.item()
    print(f"loss: {running_loss /  len(train_loader):.4f}")
        
        

Training epoch 0...
loss: 0.0015
Training epoch 1...
loss: 0.0011
Training epoch 2...
loss: 0.0011
Training epoch 3...
loss: 0.0007
Training epoch 4...
loss: 0.0008
Training epoch 5...
loss: 0.0007
Training epoch 6...
loss: 0.0005
Training epoch 7...
loss: 0.0007
Training epoch 8...
loss: 0.0007
Training epoch 9...
loss: 0.0005
Training epoch 10...
loss: 0.0006
Training epoch 11...
loss: 0.0004
Training epoch 12...
loss: 0.0003
Training epoch 13...
loss: 0.0005
Training epoch 14...
loss: 0.0007
Training epoch 15...
loss: 0.0006
Training epoch 16...
loss: 0.0004
Training epoch 17...
loss: 0.0006
Training epoch 18...
loss: 0.0004
Training epoch 19...
loss: 0.0007
Training epoch 20...
loss: 0.0006
Training epoch 21...
loss: 0.0005
Training epoch 22...
loss: 0.0004
Training epoch 23...
loss: 0.0004
Training epoch 24...
loss: 0.0002
Training epoch 25...
loss: 0.0004
Training epoch 26...
loss: 0.0004
Training epoch 27...
loss: 0.0002
Training epoch 28...
loss: 0.0001
Training epoch 29...
los

In [40]:
torch.save(net.state_dict(), 'net.pth')
net = NeuralNet()
net.load_state_dict(torch.load('net.pth'))
correct = 0 
total = 0
net.eval()
with torch.no_grad():
    for data  in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted ==  labels).sum().item()
accuracy = 100 * correct / total
print(f"Accuracy: {accuracy}")
        



Accuracy: 67.17


In [56]:
new_transform =  transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image
image_path = ['test/dog.jpg', 'test/dog.jpg', 'test/car.jpg', 'test/ship.jpeg', 'test/truck.png', 'test/bird.png' ]
images = [load_image(img) for img in image_path]
net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f"predicted {class_names[predicted.item()]}")
        

predicted dog
predicted dog
predicted car
predicted ship
predicted car
predicted bird
